In [73]:
from lxml import html
import requests
import pandas as pd
import datetime
from datedelta import datedelta
import re

In [74]:
def addMonths(date, numMonths):
    return date + datedelta(months=numMonths)

In [75]:
def addDays(date, numDays):
    return date + datedelta(days=numDays)

In [76]:
def text(elt):
    return elt.text_content().replace(u'\xa0', u' ')

In [77]:
def countRemainingDays(prev_date):
    next_year = prev_date.year + 1
    curr_date = prev_date
    while (curr_date.year < next_year):
        curr_date = curr_date + datedelta(days=1)
    temp_date = curr_date - datedelta(days=1)
    remaining_days = temp_date.day - prev_date.day + 1
    return remaining_days, curr_date

In [78]:
def getData(root, index):
    # retrieve table
    table = root.xpath('//table[@bgcolor="#d0d0d0"]')[0]

    headers = ['Date', 'Caption', 'Subcaption']

    data = []
    length = len(table.xpath('//thead/tr'))

    for i in range(2, length):
        tr = table.xpath('//thead/tr['+str(i)+']')
        date = text(tr[0].xpath('td[1]')[0])
        hour = text(tr[0].xpath('td[2]')[0])

        # merge date and time
        row_datetime = datetime.datetime.strptime(date + ' ' + hour, '%m/%d/%Y %H:%M')
        
        if(index < 18):
            image = tr[0].xpath('td[16]/img/@onmouseover')[0]
        else:
            image = tr[0].xpath('td[17]/img/@onmouseover')[0]
            
        splitImage = re.split(" overlib\('|',CAPTION,'|'\); ", image)
        row_caption = splitImage[2]
        row_subcaption = splitImage[1]

        row = []
        row.append(row_datetime)
        row.append(row_caption)
        row.append(row_subcaption)

        data.append(row)

    data = pd.DataFrame(data, columns=headers)
    return data

In [79]:
curr_date = datetime.date(2015, 1, 1)
end_of_year = False
data_list = []
for i in range(0,24):
    
    # 1. preparing the URL
    remaining_days = 50
    
    if(end_of_year == True):
        days_date = countRemainingDays(curr_date)
        remaining_days = days_date[0]
        curr_date = days_date[1]
        end_of_year = False
    else:
        curr_date = addDays(curr_date, remaining_days) 
    
    URL = 'http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=' + str(remaining_days) + '&ano=' + str(curr_date.year) + '&mes=' + str(curr_date.month) + '&day=' + str(curr_date.day) + '&hora=00'
    print(URL)
    
    # 2. get the html data from website
    page = requests.get(URL)
    root = html.fromstring(page.content)
    
    # 3. get data from html
    data = getData(root,i)
    
    if(i in [6, 14, 22]):
        i = i - 1
        end_of_year = True
        
    # 4. remove excess first row
    if(i not in [7, 15]):
        data = data.drop(data.index[0])
        
    # 5. reverse data frame order
    data = data.iloc[::-1]
        
    # 6. add to list of df
    data_list.append(data)

http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=2&day=20&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=4&day=11&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=5&day=31&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=7&day=20&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=9&day=8&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=10&day=28&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2015&mes=12&day=17&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=15&ano=2016&mes=1&day=1&hora=00
http://www.ogimet.com/cgi-bin/gsynres?lang=en&ind=98425&decoded=yes&ndays=50&ano=2016&mes=2&day=20&hora=00
http://www.ogimet.com/cgi-bin/gsynres

In [80]:
main_data = pd.concat(data_list, ignore_index=True)
main_data.to_csv('weather_ogimet_2015_2016_2017_weathercondition.csv')
main_data

,Date,Caption,Subcaption
0,2015-01-01 03:00:00,Present weather reported from a manned weather...,Haze
1,2015-01-01 06:00:00,Present weather reported from a manned weather...,"Rain, not freezing, continuous, slight at time..."
2,2015-01-01 09:00:00,Present weather reported from a manned weather...,"Rain shower(s), slight"
3,2015-01-01 12:00:00,Present weather reported from a manned weather...,Haze
4,2015-01-01 15:00:00,Present weather reported from a manned weather...,State of sky on the whole unchanged. No meteor...
5,2015-01-01 18:00:00,Present weather reported from a manned weather...,Clouds generally dissolving or becoming less d...
6,2015-01-01 21:00:00,Present weather reported from a manned weather...,Clouds generally dissolving or becoming less d...
7,2015-01-02 00:00:00,Present weather reported from a manned weather...,Haze
8,2015-01-02 03:00:00,Present weather reported from a manned weather...,Haze
9,2015-01-02 06:00:00,Present weather reported from a manned weather...,Haze


In [92]:
weather_data = pd.read_csv('weather_ogimet_2015_2016_2017.csv')

condition_data = pd.read_csv('weather_ogimet_2015_2016_2017_weathercondition.csv')
condition_data

,Date,Caption,Subcaption
0,1/1/2015 3:00,Present weather reported from a manned weather...,Haze
1,1/1/2015 6:00,Present weather reported from a manned weather...,"Rain, not freezing, continuous, slight at time..."
2,1/1/2015 9:00,Present weather reported from a manned weather...,"Rain shower(s), slight"
3,1/1/2015 12:00,Present weather reported from a manned weather...,Haze
4,1/1/2015 15:00,Present weather reported from a manned weather...,State of sky on the whole unchanged. No meteor...
5,1/1/2015 18:00,Present weather reported from a manned weather...,Clouds generally dissolving or becoming less d...
6,1/1/2015 21:00,Present weather reported from a manned weather...,Clouds generally dissolving or becoming less d...
7,1/2/2015 0:00,Present weather reported from a manned weather...,Haze
8,1/2/2015 3:00,Present weather reported from a manned weather...,Haze
9,1/2/2015 6:00,Present weather reported from a manned weather...,Haze


In [95]:
compiled_data = pd.merge(weather_data, condition_data, on='Date')
compiled_data.to_csv('weather_ogimet_2015_2016_2017_compiled.csv')